In [24]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from datetime import datetime
from urllib.request import urlretrieve

In [25]:
if 'data' not in os.listdir():
    os.mkdir('data') 

## 현재 진행중인 이벤트 크롤링

In [37]:
browser = webdriver.Chrome()

In [38]:
url = 'https://maplestory.nexon.com/News/Event/Ongoing'
browser.get(url)

In [39]:
result=[]
i=0
while True:
    event_button_list = browser.find_elements('css selector', 'div.event_board > ul >li')
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    image_list = soup.select('dt>a>img')
    for j, image in enumerate(image_list):
        urlretrieve(image['src'],f'image/event_{j}.jpg')
    event_button_list[i].click()

    j=0
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.select('p.qs_title>span')[0].text
    duration = soup.select('div.qs_info_wrap > span.event_date')[0].text
    click_num = soup.select('div.qs_info>p')[0].text
    reply_num = soup.select('div.reply_title>h2>span')[0].text
    
    while True:
        reply_button_list = browser.find_elements('css selector','div.page_numb2>a')
        reply_button_list[j].click()
        time.sleep(1)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        reply = soup.select('div.reply>p.reply_text')
        for r in reply:
            data= [title, duration, click_num, reply_num, r.text]
            result.append(data)
        j+=1
        if j==len(reply_button_list):
            break
    browser.get(url)
    i+=1
    if i==len(event_button_list):
        break
    time.sleep(1)

In [40]:
browser.quit()

In [41]:
df = pd.DataFrame(result)
df.columns = ['title', 'duration', 'click_num', 'reply_num', 'reply']
df.to_csv("data/ongoing_event.csv", index=False)

## 썬데이 메이플 크롤링

In [57]:
browser = webdriver.Chrome()

In [58]:
url = 'https://www.fmkorea.com/index.php?mid=maple&category=1135601723'
browser.get(url)

In [59]:
input_text = browser.find_elements('css selector', 'input#bd_srch_btm_itx_1135415169')[0]
drop_down = browser.find_element('css selector','span.btn_img select')
search = "썬데이 메이플(2023)"

input_text.send_keys(search)
select = Select(drop_down)
select.select_by_value('title')
input_text.send_keys(Keys.RETURN)

In [60]:
result=[]
while True:
    i=0
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    now = soup.select('strong.direction')[0].text.strip()

    while True:
        title_list= browser.find_elements('css selector','td.title.hotdeal_var8 > a' )
        time.sleep(1)
        title_list = title_list[::2]
        title_list[i].click()
        time.sleep(1)
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        now = soup.select('strong.direction')[0].text.strip()
        title = soup.select('span.np_18px_span')[0].text
        info = soup.select('span>b')
        view =info[0].text
        recom_num = info[1].text
        reply_num = info[2].text
        reply_list = soup.select('div.comment-content')
        time.sleep(1)
        for reply in reply_list:
            r = reply.text
            data = [title, view, recom_num, reply_num, r]
            result.append(data)
        i+=1
        time.sleep(1)
        if i==len(title_list):
            break
    if now=='다음':
        break
    else:
        button = browser.find_elements('css selector','a.direction')[0]
        button.click()


In [ ]:
def convert_date_format(title):
    date_str = title.split(' ')[0]
    try:
        date_obj = datetime.strptime(date_str, '%Y.%m.%d')
        new_date_str = f"{date_obj.year}년 {date_obj.month}월 {date_obj.day}일"
        return title.replace(date_str, new_date_str)
    except ValueError:
        return title

In [61]:
df = pd.DataFrame(result)
df.columns = ['title', 'view', 'recommendation number', 'reply number', 'reply']
df['title'] = df['title'].apply(convert_date_format)
df[['year', 'month', 'day']] = df['title'].str.extract(r'(\d{4})년 (\d{1,2})월 (\d{1,2})일')
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day']])
df.to_csv("data/sunday_maple_event.csv", index=False)

## 메이플 직업 크롤링

In [16]:
maple_jobs_dict ={'히어로':'혀로',
                  '팔라딘':'팔라',
                  '다크나이트':'닼나',
                  '소울마스터':'소마',
                  '미하일':'미하일',
                  '블래스터':'블래',
                  '데몬슬레이어':'데슬',
                  '데몬어벤져':'데벤',
                  '아란':'아란',
                  '카이저':'카이저',
                  '아델':'아델',
                  '제로':'제로',
                  '아크메이지(불,독)':'불독',
                  '아크메이지(썬,콜)':'선콜',
                  '비숍':'숍',
                  '플레임위자드':'플위',
                  '배틀메이지':'배메',
                  '에반':'에반',
                '루미너스':'루미',
                '일리움':'일리움',
                '라라':'라라',
                '키네시스':'키네',
                '보우마스터':'보마',
                '신궁':'신궁',
                '패스파인더':'패파',
                '윈드브레이커':'윈브',
                '와일드헌터':'와헌',
                '메르세데스':'메세',
                '카인':'카인',
                '나이트로드':'나로',
                '섀도어':'섀도어',
                '듀얼블레이드':'듀블',
                '나이트워커':'나워',
                '제논':'제논',
                '팬텀':'팬텀',
                '카데나':'카데나',
                '칼리':'칼리',
                '호영':'호영',
                '바이퍼':'바이퍼',
                '캡틴':'캡틴',
                '캐논슈터':'캐슈',
                '스트라이커':'스커',
                '메카닉':'메카닉',
                '제논':'제논',
                '은월':'은월',
                '엔젤릭버스터':'엔버',
                '아크':'아크'
                }

In [17]:
browser = webdriver.Chrome()

In [18]:
url = 'https://www.fmkorea.com/maple'
browser.get(url)

In [19]:
result=[]
drop_down = browser.find_element('css selector','span.btn_img select')
select = Select(drop_down)
select.select_by_value('title')
for key, value in maple_jobs_dict.items():
    if key==value:
        input_text = browser.find_elements('css selector', 'input#bd_srch_btm_itx_1135415169')[0]
        time.sleep(1)
        input_text.send_keys(key)
        input_text.send_keys(Keys.RETURN)
        time.sleep(1)
        url=browser.current_url
        for j in range(1, 10):
            new_url = url+f'&page={j}'
            browser.get(new_url)
            time.sleep(1)
            html = browser.page_source
            soup = BeautifulSoup(html, 'html.parser')
            time.sleep(1)
            cate_list = soup.select('td.cate')
            title_reply_list = soup.select('td.title.hotdeal_var8')
            date_list = soup.select('td.time')
            view_list = soup.select('td.m_no')[::2]
            recommend_list = soup.select('td.m_no')[1::2]
            for k in range(len(cate_list)):
                cate = cate_list[k].text.strip()
                title = title_reply_list[k].select('a.hx')[0].text.strip()
                if len(title_reply_list[k].select('a.replyNum'))==0:
                    reply = '0'
                else:
                    reply = title_reply_list[k].select('a.replyNum')[0].text.strip()
                date = date_list[k].text.strip()
                view = view_list[k].text.strip()
                if recommend_list[k].text.strip()=='':
                    recommend = '0'
                else:
                    recommend = recommend_list[k].text.strip()
                data = [key, cate, title, reply, date, view, recommend]
                result.append(data)
            if len(soup.select('p.no_doc'))!=0 and soup.select('p.no_doc')[0].text.strip()=='등록된 글이 없습니다.':
                break
        input_text = browser.find_elements('css selector', 'input#bd_srch_btm_itx_1135415169')[0]
        time.sleep(1)
        input_text.clear()
    else:
        for i in [key, value]:
            input_text = browser.find_elements('css selector', 'input#bd_srch_btm_itx_1135415169')[0]
            time.sleep(1)
            input_text.send_keys(i)
            input_text.send_keys(Keys.RETURN)
            time.sleep(1)
            url=browser.current_url
            for j in range(1, 10):
                new_url = url+f'&page={j}'
                browser.get(new_url)
                time.sleep(1)
                html = browser.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(1)
                cate_list = soup.select('td.cate')
                title_reply_list = soup.select('td.title.hotdeal_var8')
                date_list = soup.select('td.time')
                view_list = soup.select('td.m_no')[::2]
                recommend_list = soup.select('td.m_no')[1::2]
                for k in range(len(cate_list)):
                    cate = cate_list[k].text.strip()
                    title = title_reply_list[k].select('a.hx')[0].text.strip()
                    if len(title_reply_list[k].select('a.replyNum'))==0:
                        reply = '0'
                    else:
                        reply = title_reply_list[k].select('a.replyNum')[0].text.strip()
                    date = date_list[k].text.strip()
                    view = view_list[k].text.strip()
                    if recommend_list[k].text.strip()=='':
                        recommend = '0'
                    else:
                        recommend = recommend_list[k].text.strip()
                    data = [key, cate, title, reply, date, view, recommend]
                    result.append(data)
                if len(soup.select('p.no_doc'))!=0 and soup.select('p.no_doc')[0].text.strip()=='등록된 글이 없습니다.':
                    break
            input_text = browser.find_elements('css selector', 'input#bd_srch_btm_itx_1135415169')[0]
            time.sleep(1)
            input_text.clear()

In [ ]:
browser.quit()

In [28]:
df = pd.DataFrame(result)
df.columns = ['job', 'category', 'title', 'reply', 'date', 'view','recommend']
df = df.drop_duplicates()
today = datetime.today().strftime('%Y.%m.%d')
df.loc[df['date'].str.contains(':'), 'date'] = today
df.to_csv("data/jobs.csv", index=False)